In [2]:
import os
import urllib3
import json
import re
import numpy as np
import pdal
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, Point, mapping

In [3]:
MINX, MINY, MAXX, MAXY = [-93.759055, 41.925015, -93.766155, 41.935015]
polygon = Polygon(((MINX, MINY), (MINX, MAXY), (MAXX, MAXY), (MAXX, MINY), (MINX, MINY)))

In [19]:
class UsgsLidar:
    
    def __init__(self, path = "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/", pipeline_json_path: str="../pipeline.json") -> None:
            
        self.path = path
        self.input_epsg = 3857
        self.txt = self.read_txt("../data/filenames.txt")
        self.a = self.read_json("../pipeline.json")
    
    def read_json(self, json_path):
        try:
            with open(json_path) as js:
                json_obj = json.load(js)
            return json_obj

        except FileNotFoundError:
            print('File not found.')
        
    def fetch_polygon_boundaries(self, polygon: Polygon):
        polygon_df = gpd.GeoDataFrame([polygon], columns=['geometry'])

        polygon_df.set_crs(epsg=4326, inplace=True)
        polygon_df['geometry'] = polygon_df['geometry'].to_crs(epsg=3857)
        minx, miny, maxx, maxy = polygon_df['geometry'][0].bounds

        polygon_input = 'POLYGON(('
        xcords, ycords = polygon_df['geometry'][0].exterior.coords.xy
        for x, y in zip(list(xcords), list(ycords)):
            polygon_input += f'{x} {y}, '
        polygon_input = polygon_input[:-2]
        polygon_input += '))'

        return f"({[minx, maxx]},{[miny,maxy]})", polygon_input
    
    def read_csv(self, csv_path, missing_values=["n/a", "na", "undefined"]):
        try:
            df = pd.read_csv(csv_path, na_values=missing_values)
            return df

        except FileNotFoundError:
            print('File not found.')
            
    def fetch_pipeline (self, region: str, polygon: Polygon):
        url = f"{self.path}{region}/ept.json"
        boundary, poly = self.fetch_polygon_boundaries(polygon)
        
        self.a['pipeline'][0]['filename']= f"{self.path}{region}/ept.json"
        self.a['pipeline'][0]['polygon'] = poly
        self.a['pipeline'][0]['bounds'] = boundary
        pipeline = pdal.Pipeline(json.dumps(self.a))
        
        return pipeline
    
    def create_gpd_df(self, epsg):
        try:
            pipe = US.fetch_pipeline(region, polygon)
            pipe.execute()

            cloud = []
            elevations =[]
            geometry=[]
            for row in pipe.arrays[0]:
                lst = row.tolist()[-3:]
                cloud.append(lst)
                elevations.append(lst[2])
                point = Point(lst[0], lst[1])
                geometry.append(point)
            gpd_df = gpd.GeoDataFrame(columns=["elevation", "geometry"])
            gpd_df['elevation'] = elevations
            gpd_df['geometry'] = geometry
            gpd_df = gpd_df.set_geometry("geometry")
            gpd_df.set_crs(epsg = epsg, inplace=True)
            return gpd_df
        except RuntimeError as e:
            print(e)

    def read_txt(self, txt_path) -> list:
        try:
            with open(txt_path, "r") as f:
                text_file = f.read().splitlines()
            
            return text_file

        except Exception as e:
            print(e)
            
    def fetch_name_and_year(self, location: str) -> tuple:
        
        location = location.replace('/', '')
        regex = '20[0-9][0-9]+'
        match = re.search(regex, location)
        if(match):
          return (location[:match.start() - 1], location[match.start():match.end()])
        else:
          return (location, None)
    
    def fetch_metadata(self):
    
        metadata = pd.DataFrame(columns=['filename', 'region',
                          'year', 'xmin', 'xmax', 'ymin', 'ymax', 'points'])

        index = 0
        for lists in self.txt:
          r = urllib3.PoolManager().request('GET', self.path + f + "ept.json")
          if r.status == 200:
            j = json.loads(r.data)
            region, year = get_name_and_year(lists)

            metadata = metadata.append({
                'filename': lists.replace('/', ''),
                'region': region,
                'year': year,
                'xmin': j['bounds'][0],
                'xmax': j['bounds'][3],
                'ymin': j['bounds'][1],
                'ymax': j['bounds'][4],
                'points': j['points']}, ignore_index=True)

            metadata.to_csv("../data/metadata.csv")
        
        return(metadata)

In [20]:
US = UsgsLidar()
region = "IA_FullState"
df = US.read_csv("../data/iowa.csv")
shape, poly = US.fetch_polygon_boundaries(polygon)
print(poly)

POLYGON((-10437210.259858532 5149753.664381643, -10437210.259858532 5151249.971344454, -10438000.628243161 5151249.971344454, -10438000.628243161 5149753.664381643, -10437210.259858532 5149753.664381643))


In [7]:
gpd_df = US.create_gpd_df(4326)
gpd_df

,elevation,geometry
0,321.77,POINT (-93.76613 41.93170)
1,321.81,POINT (-93.76608 41.93169)
2,321.88,POINT (-93.76609 41.93169)
3,321.85,POINT (-93.76610 41.93169)
4,321.86,POINT (-93.76612 41.93170)
...,...,...
913969,307.97,POINT (-93.76172 41.93495)
913970,309.45,POINT (-93.76250 41.93496)
913971,311.15,POINT (-93.76328 41.93497)
913972,307.90,POINT (-93.76490 41.93499)


In [21]:
files = US.read_txt("../data/filenames.txt")
len(files)

1598